In [2]:
#!/usr/bin/env python3
import csv
import os
import sys
from simple_salesforce import Salesforce

# Salesforce login credentials (Replace with actual credentials if needed)
sf_username = "vlettau@everstream.net"
sf_password = "212MDcsc!!"
sf_token = "I88iovRhihZLbn0pGmYTFl5V"
sf_domain = "test"  # or 'test' for sandbox

# ---------------------------------------------------------------------
# Connect to Salesforce
# ---------------------------------------------------------------------
try:
    print("🔌 Connecting to Salesforce...")
    sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_token, domain=sf_domain)
    print(f"✅ Logged in to Salesforce instance: {sf.sf_instance}")
except Exception as e:
    print("❌ Error logging in to Salesforce:", e)
    sys.exit(1)

# ---------------------------------------------------------------------
# Paginated query parameters
# ---------------------------------------------------------------------
batch_size = 2000
offset = 0
all_records = []

print("📡 Fetching object metadata in batches...")
while True:
    query = f"""
    SELECT QualifiedApiName, Label, PluralLabel, NamespacePrefix, KeyPrefix
    FROM EntityDefinition
    ORDER BY QualifiedApiName
    LIMIT {batch_size} OFFSET {offset}
    """
    result = sf.query(query)
    records = result.get("records", [])
    all_records.extend(records)
    print(f"   ➜ Retrieved batch {offset // batch_size + 1}: {len(records)} records")

    if len(records) < batch_size:
        break
    offset += batch_size

print(f"✅ Finished fetching {len(all_records)} total objects")

# ---------------------------------------------------------------------
# Write to CSV
# ---------------------------------------------------------------------
output_file = "bbf_salesforce_objects_full.csv"
with open(output_file, mode="w", newline="", encoding="utf-8") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow([
        "QualifiedApiName",
        "Label",
        "PluralLabel",
        "NamespacePrefix",
        "KeyPrefix",
        "IsCustomObject"
    ])
    for r in all_records:
        api_name = r.get("QualifiedApiName", "")
        writer.writerow([
            api_name,
            r.get("Label", ""),
            r.get("PluralLabel", ""),
            r.get("NamespacePrefix", ""),
            r.get("KeyPrefix", ""),
            str(api_name.endswith("__c"))
        ])

print(f"📁 Export complete → {output_file}")


🔌 Connecting to Salesforce...
✅ Logged in to Salesforce instance: bluebirdnetwork--full.sandbox.my.salesforce.com
📡 Fetching object metadata in batches...
   ➜ Retrieved batch 1: 2000 records
   ➜ Retrieved batch 2: 56 records
✅ Finished fetching 2056 total objects
📁 Export complete → bbf_salesforce_objects_full.csv
